In [140]:
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt

In [141]:
def dtmf(s, a = 0.99):
    
    fs_freq = 8000
    low_freq = [697,770,852,941]
    hi_freq = [1209,1336,1477,1633]
    digits = {(941,1336): "0", (697,1209): "1", (697,1336): "2", (697,1477): "3", (770,1209): "4", (770,1336): "5", (770,1477): "6", (852,1209): "7", (852, 1336): "8", (852, 1477): "9"}
    digit_list = []
    
    for loop in [low_freq,hi_freq]:
        all = []
        
        for freq in loop:
            k = freq/fs_freq
            w = np.exp(1j*2*np.pi*k)
            y_0 = 0j
            y_1 = np.zeros(len(s),dtype = np.complex_) 
            
            for i,x in enumerate(s):
                y_1[i] = w *y_0 + x
                y_0 = y_1[i] 

            y_2 = np.square(np.abs(y_1))
            output = np.zeros(len(s)) 

            for i,x in enumerate(y_2):
                output[i] = (1-a)*x+a

            all.append(output)
        
        all = np.array(all)
        freqs = np.argmax(np.sum(all,axis=1))
        digit_list.append(np.max(freqs))
        
    digit = digits[low_freq[digit_list[0]],hi_freq[digit_list[1]]]
        
    return digit, digit_list, all

for i in range(10):
    fs_freq,s = wavfile.read(f'soundfiles/touchtone_{i}.wav')
    digit,digit_list,all = dtmf(s)
    print(f"touchtone_{i} = telphone digit:{digit}")

touchtone_0 = telphone digit:0
touchtone_1 = telphone digit:1
touchtone_2 = telphone digit:2
touchtone_3 = telphone digit:3
touchtone_4 = telphone digit:4
touchtone_5 = telphone digit:5
touchtone_6 = telphone digit:6
touchtone_7 = telphone digit:7
touchtone_8 = telphone digit:8
touchtone_9 = telphone digit:9


In [142]:
def dtmf_robust(s, a = 0.99):
    
    fs_freq = 8000
    low_freq = np.array([697,770,852,941])
    hi_freq = np.array([1209,1336,1477,1633])
    low_harm = low_freq * 2
    hi_harm = hi_freq * 2
    digits = {(941,1336): "0", (697,1209): "1", (697,1336): "2", (697,1477): "3", (770,1209): "4", (770,1336): "5", (770,1477): "6", (852,1209): "7", (852, 1336): "8", (852, 1477): "9"}
    digit_list = []

    for loop in [low_freq,low_harm,hi_freq,hi_harm]:
        all = []
        
        for freq in loop:
            k = freq/fs_freq
            w = np.exp(1j*2*np.pi*k)
            y_0 = 0j
            y_1= np.zeros(len(s),dtype = np.complex_) 
            
            for i,x in enumerate(s):
                y_1[i] = w*y_0 + x
                y_0 = y_1[i] 

            y_2 = np.square(np.abs(y_1))
            output = np.zeros(len(s)) 
            
            for i,x in enumerate(y_2):
                output[i] = (1-a)*x+a

            all.append(output)    
        
        all = np.array(all)
        freqs = np.sum(np.square(np.abs(all)),axis=1)
        digit_list.append(freqs)
        
    return digit, digit_list, all

fs_freq, s = wavfile.read('soundfiles/mystery.wav')
harmonics = [[820,2220],[2820,4220],[4820,6220],[6820,8220],[8820,10220],[10820,12220],[12820,14220],[14820,16220]]
mystery_digits = []
for i, ind in enumerate(harmonics):
    x, digit_list, all = robust_dtmfdecode(s[ind[0]:ind[1]])
    mystery_digits.append(x)
mystery_digits = [i for i in mystery_digits if i is not None]
print(f"The Mystery Digits are {mystery_digits}")


The Mystery Digits are ['8', '6', '7', '5', '3', '0', '9']
